#### Домашнее задание по теме «Рекомендации на основе содержания»

* Использовать dataset MovieLens
* Построить рекомендации (регрессия, предсказываем оценку) на фичах:
* TF-IDF на тегах и жанрах
* Средние оценки (+ median, variance, etc.) пользователя и фильма
* Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
links = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\links.csv')
movies = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\movies.csv')
ratings = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\ratings.csv')
tags = pd.read_csv(r'C:\Users\Urmas\Desktop\Digital\НЕТОЛОГИЯ. ПРОФЕССИЯ DATA SCIENTIST\Рекомендательные системы\Рекомендации на основе содержания\tags.csv')

In [3]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [7]:
data1 = movies.join(tags.set_index('movieId'), on='movieId')    # movies.merge(tags)
data1.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


In [8]:
data2 = data1.join(ratings.set_index('movieId'), on='movieId', lsuffix='_left', rsuffix='_right')
# data1.merge(ratings.set_index('movieId'), on='movieId')
data2.head()

,movieId,title,genres,userId_left,tag,timestamp_left,userId_right,rating,timestamp_right
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,1.0,4.0,9.649827e+08
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,5.0,4.0,8.474350e+08
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,7.0,4.5,1.106636e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,15.0,2.5,1.510578e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09,17.0,4.5,1.305696e+09


In [9]:
data = data2[['movieId', 'genres', 'userId_left', 'tag', 'userId_right', 'rating']]
data.head()

,movieId,genres,userId_left,tag,userId_right,rating
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5


In [10]:
data['Genre'] = data['genres'].apply(lambda x: ' '.join(x.replace(' ', '').replace('-', '').split('|')))
data.head()

<ipython-input-10-632bfc514edd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Genre'] = data['genres'].apply(lambda x: ' '.join(x.replace(' ', '').replace('-', '').split('|')))


,movieId,genres,userId_left,tag,userId_right,rating,Genre
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285783 entries, 0 to 9741
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   movieId       285783 non-null  int64  
 1   genres        285783 non-null  object 
 2   userId_left   233234 non-null  float64
 3   tag           233234 non-null  object 
 4   userId_right  285762 non-null  float64
 5   rating        285762 non-null  float64
 6   Genre         285783 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 17.4+ MB


In [12]:
data.dropna(inplace=True)

<ipython-input-12-f1116dacf2bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233213 entries, 0 to 9732
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   movieId       233213 non-null  int64  
 1   genres        233213 non-null  object 
 2   userId_left   233213 non-null  float64
 3   tag           233213 non-null  object 
 4   userId_right  233213 non-null  float64
 5   rating        233213 non-null  float64
 6   Genre         233213 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 14.2+ MB


##### Выведем метрику на каждый фильм и добавим его в фичи

In [14]:
# посчитаем среднюю оценку на каждый фильм
mean_score_mov = {}

for title, group in tqdm(data.groupby('movieId')):
    mean_score_mov[title] = group.rating.mean()

In [15]:
# посчитаем по каждому фильму количество рейтингов
num_action_mov = {}

for title, group in tqdm(data.groupby('movieId')):
    num_action_mov[title] = group.shape[0]

In [16]:
# метрики
min_num_action_mov = np.min([num_action_mov[f] for f in num_action_mov.keys()])
max_num_action_mov = np.max([num_action_mov[f] for f in num_action_mov.keys()])
mean_num_action_mov = np.mean([num_action_mov[f] for f in num_action_mov.keys()])

In [17]:
# посчитаем метрику фильма
score_mov = {}

for f in num_action_mov.keys():
        score_mov[f] = (mean_score_mov[f] * (num_action_mov[f] - mean_num_action_mov)/
                        (max_num_action_mov - min_num_action_mov))

In [18]:
# добавим признак - метрику фильма

data['score_mov'] = data['movieId'].apply(lambda x: score_mov[x])
data.head()

<ipython-input-18-560b0b65f2e7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['score_mov'] = data['movieId'].apply(lambda x: score_mov[x])


,movieId,genres,userId_left,tag,userId_right,rating,Genre,score_mov
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy,0.034924
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924


##### Посчитаем среднюю оценку каждого userId и добавим в фичи

In [19]:
mean_mark = {}

for title, group in tqdm(data.groupby('userId_right')):
    mean_mark[title] = group.rating.mean()

In [20]:
data['mean_mark'] = data['userId_right'].apply(lambda x: mean_mark[x])
data.head()

<ipython-input-20-64cb55d52633>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['mean_mark'] = data['userId_right'].apply(lambda x: mean_mark[x])


,movieId,genres,userId_left,tag,userId_right,rating,Genre,score_mov,mean_mark
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.040472
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.517986
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,3.672316
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy,0.034924,3.601810
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,4.538404


In [21]:
Tag = {}
for title, group in tqdm(data.groupby('movieId')):
    Tag[title] = list(group.tag.unique())
Tag[1]

['pixar', 'fun']

In [22]:
data['Tag'] = data['movieId'].apply(lambda x: str(Tag[x]).strip("['']"))
data['Tag'] = data['Tag'].apply(lambda x: re.sub("'", '', x))
data['Tag'] = data['Tag'].apply(lambda x: re.sub(',', '', x))
data.head()

<ipython-input-22-92d2fa6434f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tag'] = data['movieId'].apply(lambda x: str(Tag[x]).strip("['']"))
<ipython-input-22-92d2fa6434f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Tag'] = data['Tag'].apply(lambda x: re.sub("'", '', x))
<ipython-input-22-92d2fa6434f4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,movieId,genres,userId_left,tag,userId_right,rating,Genre,score_mov,mean_mark,Tag
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.040472,pixar fun
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.517986,pixar fun
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,3.672316,pixar fun
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy,0.034924,3.601810,pixar fun
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,4.538404,pixar fun


In [23]:
tfidf = TfidfVectorizer()

In [24]:
tfidf.fit(data['Genre'])

TfidfVectorizer()

In [25]:
data['tfidf_Genre'] = data['Genre'].apply(lambda x: tfidf.transform(list(x)).toarray())
data.head()

<ipython-input-25-6ab647c008b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tfidf_Genre'] = data['Genre'].apply(lambda x: tfidf.transform(list(x)).toarray())


,movieId,genres,userId_left,tag,userId_right,rating,Genre,score_mov,mean_mark,Tag,tfidf_Genre
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.040472,pixar fun,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.517986,pixar fun,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,3.672316,pixar fun,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy,0.034924,3.601810,pixar fun,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,4.538404,pixar fun,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [26]:
tfidf2 = TfidfVectorizer()

In [27]:
Tag2 = []
for i, j in Tag.items():
    #Tag2.append(re.sub("'", '', str(j).strip("['']")))
    Tag2.append(str(j).strip("['']"))


tfidf2.fit(Tag2)

tfidf_Tag = {}

for title, group in tqdm(Tag.items()):
    tfidf_Tag[title] = tfidf2.transform(group).toarray()
tfidf_Tag[1]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
data['tfidf_Tag']= data['movieId'].apply(lambda x: tfidf_Tag[x])
data.head()

<ipython-input-28-4d6f817c5e9e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tfidf_Tag']= data['movieId'].apply(lambda x: tfidf_Tag[x])


,movieId,genres,userId_left,tag,userId_right,rating,Genre,score_mov,mean_mark,Tag,tfidf_Genre,tfidf_Tag
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.040472,pixar fun,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,5.0,4.0,Adventure Animation Children Comedy Fantasy,0.034924,4.517986,pixar fun,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,7.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,3.672316,pixar fun,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,15.0,2.5,Adventure Animation Children Comedy Fantasy,0.034924,3.601810,pixar fun,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1,Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,17.0,4.5,Adventure Animation Children Comedy Fantasy,0.034924,4.538404,pixar fun,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [29]:
data = data[['userId_right', 'rating', 'score_mov', 'mean_mark', 'tfidf_Genre', 'tfidf_Tag']]
data.head()

,userId_right,rating,score_mov,mean_mark,tfidf_Genre,tfidf_Tag
0,1.0,4.0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,5.0,4.0,0.034924,4.517986,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,7.0,4.5,0.034924,3.672316,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,15.0,2.5,0.034924,3.601810,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,17.0,4.5,0.034924,4.538404,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [30]:
data_X = data[data['userId_right']==1]
data_X.head()

,userId_right,rating,score_mov,mean_mark,tfidf_Genre,tfidf_Tag
0,1.0,4.0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1.0,4.0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,1.0,4.0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,1.0,4.0,-0.002703,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,1.0,4.0,-0.002703,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [163]:
X = data_X.drop(columns=['userId_right', 'rating'])
X.head()

,score_mov,mean_mark,tfidf_Genre,tfidf_Tag
0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,-0.002703,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,-0.002703,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [164]:
#X['concat'] = X.apply(lambda x: np.insert(np.concatenate((X['tfidf_Genre'], X['tfidf_Tag']), axis=0).T, [0,1], [X['score_mov'], X['mean_mark']]), axis=1)
#X['concat'] = X.apply(lambda x: np.concatenate((x['tfidf_Genre'], x['tfidf_Tag']), axis=0))
#X['concat'] = X.apply(lambda с: np.concatenate((np.array(с['tfidf_Genre']), np.array(с['tfidf_Tag'])), axis=0).T, axis=1)
#X['concat'] = X.apply(lambda с: np.concatenate((с['tfidf_Genre'].ravel(), с['tfidf_Tag'].ravel()), axis=0).T, axis=1)
X['concat'] = X.apply(lambda с: np.concatenate((с['tfidf_Genre'].ravel(), с['tfidf_Tag'].ravel()), axis=0).T, axis=1)
X['concat2'] = X.apply(lambda c: np.concatenate((np.array([c['score_mov']]), np.array([c['mean_mark']])), axis=0), axis=1)
X.head()

,score_mov,mean_mark,tfidf_Genre,tfidf_Tag,concat,concat2
0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.03492379067107298, 4.040472175379427]"
0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.03492379067107298, 4.040472175379427]"
0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.03492379067107298, 4.040472175379427]"
2,-0.002703,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0027027198735167634, 4.040472175379427]"
2,-0.002703,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0027027198735167634, 4.040472175379427]"


In [165]:
X['Conc'] = X.apply(lambda x: np.concatenate((x['concat'].ravel(), x['concat2'].ravel()), axis=0).ravel(), axis=1)
X.head()

,score_mov,mean_mark,tfidf_Genre,tfidf_Tag,concat,concat2,Conc
0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.03492379067107298, 4.040472175379427]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.03492379067107298, 4.040472175379427]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
0,0.034924,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.03492379067107298, 4.040472175379427]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,-0.002703,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0027027198735167634, 4.040472175379427]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,-0.002703,4.040472,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0027027198735167634, 4.040472175379427]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [166]:
X = X[['Conc']].values

In [167]:
X.shape

(593, 1)

In [168]:
X = np.array(X.tolist())[:,-1,:]

<ipython-input-168-3808cbfb2129>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array(X.tolist())[:,-1,:]


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [93]:
y = data_X['rating']

In [121]:
lr = LinearRegression()

In [169]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [170]:
lr.fit(X_train, y_train)

ValueError: setting an array element with a sequence.

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = mean_squared_error(y_test, y_pred, squared=False)
#rmse = sqrt(mean_squared_error(y_test, y_pred))